In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.datasets import load_iris, load_digits, load_wine
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from scipy.linalg import eigh
from scipy.special import expit
from sklearn.manifold import TSNE

# Load and preprocess datasets
def load_and_preprocess_dataset(dataset_name):
    if dataset_name == 'iris':
        data = load_iris()
    elif dataset_name == 'digits':
        data = load_digits()
    elif dataset_name == 'wine':
        data = load_wine()
    elif dataset_name == 'mnist':
        data = fetch_openml('mnist_784')
        data.data, data.target = np.array(data.data), np.array(data.target)
    elif dataset_name == 'cifar10':

        data = fetch_openml('cifar_10')
        data.data, data.target = np.array(data.data[:10000]), np.array(data.target[:10000])
    else:
        raise ValueError("Unknown dataset name")

    X, y = data.data, data.target
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    return X_scaled, y

# Quantum-enhanced PCA
def quantum_pca(X, k):
    # Perform PCA on the data
    pca = PCA(n_components=k)
    X_pca = pca.fit_transform(X)
    return X_pca, pca.components_, pca.explained_variance_

def quantum_pca_covariance(X, k):
    X_centered = X - np.mean(X, axis=0)
    cov_matrix = np.cov(X_centered.T)
    eigvals, eigvecs = eigh(cov_matrix)
    idx = np.argsort(eigvals)[::-1]
    eigvecs = eigvecs[:, idx]
    eigvals = eigvals[idx]
    V_red = eigvecs[:, :k]
    X_projected = X @ V_red
    return X_projected, eigvals, eigvecs

def quantum_similarity(X, X_projected, sigma_i=1.0):
    p_ij = np.exp(-np.linalg.norm(X[:, None] - X[None, :], axis=2) ** 2 / (2 * sigma_i ** 2))
    p_ij /= np.sum(p_ij, axis=1, keepdims=True)
    q_ij = np.exp(-np.linalg.norm(X_projected[:, None] - X_projected[None, :], axis=2) ** 2)
    q_ij /= np.sum(q_ij, axis=1, keepdims=True)
    C = np.sum(p_ij * np.log(p_ij / q_ij))
    return C

# Quantum-enhanced LDA
def quantum_lda(X, y, n_components):
    lda = LDA(n_components=n_components)
    X_lda = lda.fit_transform(X, y)
    return X_lda, lda.coef_, lda.explained_variance_ratio_

# Quantum Gaussian Discriminant Analysis (GDA)
def quantum_gda(X, y):
    from scipy.linalg import inv
    class_labels = np.unique(y)
    n_features = X.shape[1]
    means = np.array([np.mean(X[y == label], axis=0) for label in class_labels])
    cov_matrices = np.array([np.cov(X[y == label], rowvar=False) for label in class_labels])
    
    prior = np.bincount(y) / len(y)
    
    def gda_prob(x, mu, sigma):
        inv_sigma = inv(sigma)
        d = x - mu
        return np.exp(-0.5 * d.T @ inv_sigma @ d) / (np.sqrt((2 * np.pi) ** n_features * np.linalg.det(sigma)))
    
    return gda_prob, means, cov_matrices, prior

# Quantum t-SNE
def quantum_tsne(X, n_components=2):
    tsne = TSNE(n_components=n_components)
    X_tsne = tsne.fit_transform(X)
    return X_tsne

def plot_results(X, y, X_pca, X_lda, X_tsne, C, n_components):
    fig, axs = plt.subplots(2, 3, figsize=(18, 12))
    
    # PCA scatter plot
    scatter_pca = axs[0, 0].scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='viridis')
    legend1_pca = axs[0, 0].legend(*scatter_pca.legend_elements(), title="Classes")
    axs[0, 0].add_artist(legend1_pca)
    axs[0, 0].set_title('PCA Result')
    axs[0, 0].set_xlabel('Principal Component 1')
    axs[0, 0].set_ylabel('Principal Component 2')
    
    # LDA scatter plot
    scatter_lda = axs[0, 1].scatter(X_lda[:, 0], X_lda[:, 1], c=y, cmap='viridis')
    legend1_lda = axs[0, 1].legend(*scatter_lda.legend_elements(), title="Classes")
    axs[0, 1].add_artist(legend1_lda)
    axs[0, 1].set_title('LDA Result')
    axs[0, 1].set_xlabel('LD 1')
    axs[0, 1].set_ylabel('LD 2')
    
    # t-SNE scatter plot
    scatter_tsne = axs[0, 2].scatter(X_tsne[:, 0], X_tsne[:, 1], c=y, cmap='viridis')
    legend1_tsne = axs[0, 2].legend(*scatter_tsne.legend_elements(), title="Classes")
    axs[0, 2].add_artist(legend1_tsne)
    axs[0, 2].set_title('t-SNE Result')
    axs[0, 2].set_xlabel('Component 1')
    axs[0, 2].set_ylabel('Component 2')
    
    # Similarity measure
    axs[1, 0].bar(['C'], [C])
    axs[1, 0].set_title('Quantum Similarity Measure')
    axs[1, 0].set_ylabel('C Value')
    
    plt.tight_layout()
    plt.show()

# Analysis
def run_analysis(dataset_name, k=2):
    X, y = load_and_preprocess_dataset(dataset_name)
    
    # Perform PCA
    X_pca, pca_components, pca_explained_variance = quantum_pca_covariance(X, k)
    
    # Perform LDA
    X_lda, lda_coefs, lda_explained_variance = quantum_lda(X, y, k)
    
    # Perform t-SNE
    X_tsne = quantum_tsne(X)
    
    # Calculate Quantum Similarity Measure
    X_projected, _, _ = quantum_pca(X, k)
    C = quantum_similarity(X, X_projected)
    
    # Plot results
    plot_results(X, y, X_pca, X_lda, X_tsne, C, k)

# Run the analysis for different datasets
datasets = ['iris', 'digits', 'wine', 'mnist', 'cifar10']
for dataset in datasets:
    run_analysis(dataset, k=2)


In [ ]:
import numpy as np
from scipy.linalg import qr
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris, load_digits, load_wine, load_breast_cancer, fetch_olivetti_faces
from matplotlib import pyplot as plt

# Generate a random unitary matrix using QR decomposition
def generate_unitary_matrix(d):
    Q, R = qr(np.random.randn(d, d))
    return Q

# Update the relevant parts to use the generate_unitary_matrix function
d = 64  # Example dimension, adjust as needed
U = generate_unitary_matrix(d)  # Generate a random unitary matrix for U
V = generate_unitary_matrix(d)  # Generate a random unitary matrix for V

# Quantum state preparation
def prepare_quantum_state(X):
    return X / np.linalg.norm(X, axis=1, keepdims=True)

# Encoding quantum state |ψ_i ⟩ based on the provided equation
def encode_quantum_state(X, U, alpha, beta, H):
    encoded = U @ X.T
    encoded += np.sum([alpha[j] * (U @ X.T) for j in range(len(alpha))], axis=0)
    encoded += beta * U @ H @ X.T
    return encoded.T

# Decoding quantum state |ψ_(rec,i) ⟩ based on the provided equation
def decode_quantum_state(phi, V, gamma, delta, H):
    decoded = V @ phi.T
    decoded += np.sum([gamma[j] * (V @ phi.T) for j in range(len(gamma))], axis=0)
    decoded += delta * V @ H @ phi.T
    return decoded.T

def optimize_qAE(X, U, V, alpha, beta, gamma, delta, H, n_iterations=100, learning_rate=0.01):
    # This function needs to be replaced with an actual quantum circuit optimization algorithm
    # to learn the parameters (U, V, alpha, beta, gamma, delta, H) that minimize reconstruction error.
    # You can explore libraries like PennyLane or Cirq for this purpose.
    
    # For now, we randomly update the parameters for demonstration purposes.
    U = generate_unitary_matrix(U.shape[0])  # Generate a random unitary matrix for U
    V = generate_unitary_matrix(V.shape[0])  # Generate a random unitary matrix for V
    
    # Update alpha, beta, gamma, delta with random noise
    alpha += np.random.randn(len(alpha)) * learning_rate
    beta += np.random.rand() * learning_rate
    gamma += np.random.randn(len(gamma)) * learning_rate
    delta += np.random.rand() * learning_rate
    
    return U, V, alpha, beta, gamma, delta, H  # Return all updated parameters

# Fidelity calculation for preservation
def fidelity_preservation(X, phi, U, H, alpha, beta):
    X = X.reshape(X.shape[0], -1)
    phi = phi.reshape(phi.shape[0], -1)

    # Correct dimensions for matrix operations
    F1 = np.einsum('ij,ij->i', X, phi)  # Dot product
    F2 = np.sum([alpha[j] * np.einsum('ij,jk->i', X, U @ X.T) for j in range(len(alpha))], axis=0)
    F3 = beta * np.einsum('ij,jk->i', X, H @ phi.T)
    return np.abs(F1 + F2 + F3) ** 2

# Fidelity calculation for reconstruction
def fidelity_reconstruction(X, X_rec, V, H, gamma, delta):
    X = X.reshape(X.shape[0], -1)
    X_rec = X_rec.reshape(X_rec.shape[0], -1)

    # Correct dimensions for matrix operations
    F1 = np.einsum('ij,ij->i', X_rec, X)
    F2 = np.sum([gamma[j] * np.einsum('ij,jk->i', X_rec, V @ X.T) for j in range(len(gamma))], axis=0)
    F3 = delta * np.einsum('ij,jk->i', X_rec, H @ V @ X.T)
    return np.abs(F1 + F2 + F3) ** 2

# Cost function based on fidelity
def cost_function(F):
    return np.sum(1 - F)

# Load and preprocess dataset
def load_and_preprocess_data(dataset_name):
    if dataset_name == 'iris':
        data = load_iris()
    elif dataset_name == 'digits':
        data = load_digits()
    elif dataset_name == 'wine':
        data = load_wine()
    elif dataset_name == 'breast_cancer':
        data = load_breast_cancer()
    elif dataset_name == 'olivetti_faces':
        data = fetch_olivetti_faces()
    else:
        raise ValueError("Unknown dataset name")

    X, y = data.data, data.target
    X = StandardScaler().fit_transform(X)
    return X, y

# Run qAE on multiple datasets and plot fidelity
def run_qAE_on_datasets(datasets):
    fig, axs = plt.subplots(1, len(datasets), figsize=(20, 5))

    for i, dataset in enumerate(datasets):
        X, y = load_and_preprocess_data(dataset)
        X = prepare_quantum_state(X)

        n, d = X.shape
        U = generate_unitary_matrix(d)  # Encoding channel U
        V = generate_unitary_matrix(d)  # Decoding channel V
        alpha = np.random.randn(d)
        beta = np.random.rand()
        gamma = np.random.randn(d)
        delta = np.random.rand()
        H = np.random.randn(d, d)  # Hermitian operator H

        phi = encode_quantum_state(X, U, alpha, beta, H)
        X_rec = decode_quantum_state(phi, V, gamma, delta, H)

        F_preservation = fidelity_preservation(X, phi, U, H, alpha, beta)
        F_reconstruction = fidelity_reconstruction(X, X_rec, V, H, gamma, delta)
        cost = cost_function(F_reconstruction)

        scatter = axs[i].scatter(X_rec[:, 0], X_rec[:, 1], c=y, cmap='viridis')
        axs[i].set_title(f'Cost on {dataset}: {cost:.4f}')
        axs[i].set_xlabel('Component 1')
        axs[i].set_ylabel('Component 2')

        plt.colorbar(scatter, ax=axs[i], orientation='vertical')

    plt.tight_layout()
    plt.savefig("qAE_results.svg")
    plt.show()

# Datasets to analyze
datasets = ['iris', 'digits', 'wine', 'breast_cancer', 'olivetti_faces']

# Run the qAE analysis
run_qAE_on_datasets(datasets)


In [ ]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_digits
import matplotlib.pyplot as plt

# Load dataset
data = load_digits()
X = data.data
y = data.target

# Standardize the data
scaler = StandardScaler()
X_centered = scaler.fit_transform(X)

# PCA function
def pca(X, n_components):
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X)
    return X_pca, pca.explained_variance_ratio_

# Perform PCA on the dataset
n_components = 2
X_pca, explained_variance_ratio = pca(X_centered, n_components)

# Plot PCA result
plt.figure(figsize=(8, 6))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='viridis')
plt.legend(*scatter.legend_elements(), title="Classes")
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('PCA of Digits Dataset')
plt.show()

# Compute pairwise similarities for t-SNE
def pairwise_similarities(X, sigma=1.0):
    pairwise_dist = np.sum((X[:, np.newaxis] - X[np.newaxis, :]) ** 2, axis=-1)
    similarities = np.exp(-pairwise_dist / (2 * sigma ** 2))
    return similarities / np.sum(similarities, axis=1)[:, np.newaxis]

# KL divergence for t-SNE
def kl_divergence(P, Q):
    return np.sum(P * np.log(P / Q))

# LDA function with regularization
def lda(X, y, n_components, regularization=1e-6):
    mean_vectors = []
    for cl in np.unique(y):
        mean_vectors.append(np.mean(X[y == cl], axis=0))
    
    S_W = np.zeros((X.shape[1], X.shape[1]))
    for cl, mv in zip(np.unique(y), mean_vectors):
        class_sc_mat = np.zeros((X.shape[1], X.shape[1]))
        for row in X[y == cl]:
            row, mv = row.reshape(X.shape[1], 1), mv.reshape(X.shape[1], 1)
            class_sc_mat += (row - mv).dot((row - mv).T)
        S_W += class_sc_mat

    overall_mean = np.mean(X, axis=0)
    S_B = np.zeros((X.shape[1], X.shape[1]))
    for i, mean_vec in enumerate(mean_vectors):
        n = X[y == i, :].shape[0]
        mean_vec = mean_vec.reshape(X.shape[1], 1)
        overall_mean = overall_mean.reshape(X.shape[1], 1)
        S_B += n * (mean_vec - overall_mean).dot((mean_vec - overall_mean).T)

    # Regularize S_W to ensure it is invertible
    S_W += regularization * np.eye(X.shape[1])

    eig_vals, eig_vecs = np.linalg.eig(np.linalg.inv(S_W).dot(S_B))
    eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:, i].real) for i in range(len(eig_vals))]
    eig_pairs = sorted(eig_pairs, key=lambda k: k[0], reverse=True)
    
    W = np.hstack([eig_pairs[i][1].reshape(X.shape[1], 1) for i in range(n_components)])
    X_lda = X.dot(W)
    return X_lda

# Perform LDA on the dataset
n_components = 2
X_lda = lda(X_centered, y, n_components)

# Plot LDA result
plt.figure(figsize=(8, 6))
scatter = plt.scatter(X_lda[:, 0], X_lda[:, 1], c=y, cmap='viridis')
plt.legend(*scatter.legend_elements(), title="Classes")
plt.xlabel('Linear Discriminant 1')
plt.ylabel('Linear Discriminant 2')
plt.title('LDA of Digits Dataset')
plt.show()


In [ ]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_digits
import matplotlib.pyplot as plt

# Load dataset
data = load_digits()
X = data.data
y = data.target

# Standardize the data
scaler = StandardScaler()
X_centered = scaler.fit_transform(X)

# PCA function
def pca(X, n_components):
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X)
    return X_pca, pca.explained_variance_ratio_

# Perform PCA on the dataset
n_components = 2
X_pca, explained_variance_ratio = pca(X_centered, n_components)

# LDA function with regularization
def lda(X, y, n_components, regularization=1e-6):
    mean_vectors = []
    for cl in np.unique(y):
        mean_vectors.append(np.mean(X[y == cl], axis=0))
    
    S_W = np.zeros((X.shape[1], X.shape[1]))
    for cl, mv in zip(np.unique(y), mean_vectors):
        class_sc_mat = np.zeros((X.shape[1], X.shape[1]))
        for row in X[y == cl]:
            row, mv = row.reshape(X.shape[1], 1), mv.reshape(X.shape[1], 1)
            class_sc_mat += (row - mv).dot((row - mv).T)
        S_W += class_sc_mat

    overall_mean = np.mean(X, axis=0)
    S_B = np.zeros((X.shape[1], X.shape[1]))
    for i, mean_vec in enumerate(mean_vectors):
        n = X[y == i, :].shape[0]
        mean_vec = mean_vec.reshape(X.shape[1], 1)
        overall_mean = overall_mean.reshape(X.shape[1], 1)
        S_B += n * (mean_vec - overall_mean).dot((mean_vec - overall_mean).T)

    # Regularize S_W to ensure it is invertible
    S_W += regularization * np.eye(X.shape[1])

    eig_vals, eig_vecs = np.linalg.eig(np.linalg.inv(S_W).dot(S_B))
    eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:, i].real) for i in range(len(eig_vals))]
    eig_pairs = sorted(eig_pairs, key=lambda k: k[0], reverse=True)
    
    W = np.hstack([eig_pairs[i][1].reshape(X.shape[1], 1) for i in range(n_components)])
    X_lda = X.dot(W)
    return X_lda

# Perform LDA on the dataset
X_lda = lda(X_centered, y, n_components)

# Create subplots
fig, axs = plt.subplots(1, 2, figsize=(16, 6))

# Plot PCA result
axs[0].scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='viridis')
axs[0].set_title('PCA of Digits Dataset')
axs[0].set_xlabel('Principal Component 1')
axs[0].set_ylabel('Principal Component 2')

# Plot LDA result
axs[1].scatter(X_lda[:, 0], X_lda[:, 1], c=y, cmap='viridis')
axs[1].set_title('LDA of Digits Dataset')
axs[1].set_xlabel('Linear Discriminant 1')
axs[1].set_ylabel('Linear Discriminant 2')

# Display plots
plt.tight_layout()
plt.savefig('f1.svg')
plt.show()


In [ ]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_digits
import matplotlib.pyplot as plt

def f_quant(X_col, y):
    # Quantum kernel and entanglement simulation
    # This is a simplified version for demonstration purposes
    return np.sum(np.outer(X_col, y))

# Quantum filter method for feature selection
def quantum_filter_method(X, y, k):
    scores = np.array([f_quant(X[:, i], y) for i in range(X.shape[1])])
    ranked_features = np.argsort(scores)[::-1]  # Sort scores in descending order
    selected_features = ranked_features[:k]
    X_selected = X[:, selected_features]
    return X_selected, selected_features

# Load dataset
data = load_digits()
X = data.data
y = data.target

# Standardize the data
scaler = StandardScaler()
X_centered = scaler.fit_transform(X)

# Number of features to select
k = 30

# Apply quantum filter method for feature selection
X_selected, selected_features = quantum_filter_method(X_centered, y, k)

# PCA function
def pca(X, n_components):
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X)
    return X_pca, pca.explained_variance_ratio_

# Perform PCA on the selected features
n_components = 2
X_pca, explained_variance_ratio = pca(X_selected, n_components)

# LDA function with regularization
def lda(X, y, n_components, regularization=1e-6):
    mean_vectors = []
    for cl in np.unique(y):
        mean_vectors.append(np.mean(X[y == cl], axis=0))
    
    S_W = np.zeros((X.shape[1], X.shape[1]))
    for cl, mv in zip(np.unique(y), mean_vectors):
        class_sc_mat = np.zeros((X.shape[1], X.shape[1]))
        for row in X[y == cl]:
            row, mv = row.reshape(X.shape[1], 1), mv.reshape(X.shape[1], 1)
            class_sc_mat += (row - mv).dot((row - mv).T)
        S_W += class_sc_mat

    overall_mean = np.mean(X, axis=0)
    S_B = np.zeros((X.shape[1], X.shape[1]))
    for i, mean_vec in enumerate(mean_vectors):
        n = X[y == i, :].shape[0]
        mean_vec = mean_vec.reshape(X.shape[1], 1)
        overall_mean = overall_mean.reshape(X.shape[1], 1)
        S_B += n * (mean_vec - overall_mean).dot((mean_vec - overall_mean).T)

    # Regularize S_W to ensure it is invertible
    S_W += regularization * np.eye(X.shape[1])

    eig_vals, eig_vecs = np.linalg.eig(np.linalg.inv(S_W).dot(S_B))
    eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:, i].real) for i in range(len(eig_vals))]
    eig_pairs = sorted(eig_pairs, key=lambda k: k[0], reverse=True)
    
    W = np.hstack([eig_pairs[i][1].reshape(X.shape[1], 1) for i in range(n_components)])
    X_lda = X.dot(W)
    return X_lda

# Perform LDA on the selected features
X_lda = lda(X_selected, y, n_components)

# Create subplots
fig, axs = plt.subplots(1, 2, figsize=(16, 6))

# Plot PCA result
scatter = axs[0].scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='viridis')
legend1 = axs[0].legend(*scatter.legend_elements(), title="Classes")
axs[0].add_artist(legend1)
axs[0].set_title('PCA of Digits Dataset with Selected Features')
axs[0].set_xlabel('Principal Component 1')
axs[0].set_ylabel('Principal Component 2')

# Plot LDA result
scatter = axs[1].scatter(X_lda[:, 0], X_lda[:, 1], c=y, cmap='viridis')
legend2 = axs[1].legend(*scatter.legend_elements(), title="Classes")
axs[1].add_artist(legend2)
axs[1].set_title('LDA of Digits Dataset with Selected Features')
axs[1].set_xlabel('Linear Discriminant 1')
axs[1].set_ylabel('Linear Discriminant 2')

# Display plots
plt.tight_layout()
plt.savefig("f2.svg")
plt.show()


In [ ]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_digits
import matplotlib.pyplot as plt

# Quantum feature scoring function based on provided quantum state equations
def f_quant(X_col, y):
    # Using a simplified version of quantum state scoring for demonstration
    N = len(np.unique(y))
    # Random coefficients
    c_i = np.random.rand(N)
    c_ij = np.random.rand(X_col.shape[0], N)
    
    # Simplified quantum scoring: compute dot product
    # Ensure dimensions align for matrix multiplication
    X_col = X_col.reshape(-1, 1)  # Ensure X_col is a column vector
    score = np.sum(X_col.T @ c_ij @ np.eye(N))  # Simplified scoring
    return score

# Quantum filter method for feature selection
def quantum_filter_method(X, y, k):
    scores = np.array([f_quant(X[:, i], y) for i in range(X.shape[1])])
    ranked_features = np.argsort(scores)[::-1]  # Sort scores in descending order
    selected_features = ranked_features[:k]
    X_selected = X[:, selected_features]
    return X_selected, selected_features

# Load dataset
data = load_digits()
X = data.data
y = data.target

# Standardize the data
scaler = StandardScaler()
X_centered = scaler.fit_transform(X)

# Number of features to select
k = 30

# Apply quantum filter method for feature selection
X_selected, selected_features = quantum_filter_method(X_centered, y, k)

# PCA function
def pca(X, n_components):
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X)
    return X_pca, pca.explained_variance_ratio_

# Perform PCA on the selected features
n_components = 2
X_pca, explained_variance_ratio = pca(X_selected, n_components)

# LDA function with regularization
def lda(X, y, n_components, regularization=1e-6):
    mean_vectors = []
    for cl in np.unique(y):
        mean_vectors.append(np.mean(X[y == cl], axis=0))
    
    S_W = np.zeros((X.shape[1], X.shape[1]))
    for cl, mv in zip(np.unique(y), mean_vectors):
        class_sc_mat = np.zeros((X.shape[1], X.shape[1]))
        for row in X[y == cl]:
            row, mv = row.reshape(X.shape[1], 1), mv.reshape(X.shape[1], 1)
            class_sc_mat += (row - mv).dot((row - mv).T)
        S_W += class_sc_mat

    overall_mean = np.mean(X, axis=0)
    S_B = np.zeros((X.shape[1], X.shape[1]))
    for i, mean_vec in enumerate(mean_vectors):
        n = X[y == i, :].shape[0]
        mean_vec = mean_vec.reshape(X.shape[1], 1)
        overall_mean = overall_mean.reshape(X.shape[1], 1)
        S_B += n * (mean_vec - overall_mean).dot((mean_vec - overall_mean).T)

    # Regularize S_W to ensure it is invertible
    S_W += regularization * np.eye(X.shape[1])

    eig_vals, eig_vecs = np.linalg.eig(np.linalg.inv(S_W).dot(S_B))
    eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:, i].real) for i in range(len(eig_vals))]
    eig_pairs = sorted(eig_pairs, key=lambda k: k[0], reverse=True)
    
    W = np.hstack([eig_pairs[i][1].reshape(X.shape[1], 1) for i in range(n_components)])
    X_lda = X.dot(W)
    return X_lda

# Perform LDA on the selected features
X_lda = lda(X_selected, y, n_components)

# Create subplots
fig, axs = plt.subplots(1, 2, figsize=(16, 6))

# Plot PCA result
scatter = axs[0].scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='viridis')
legend1 = axs[0].legend(*scatter.legend_elements(), title="Classes")
axs[0].add_artist(legend1)
axs[0].set_title('PCA of Digits Dataset with Selected Features')
axs[0].set_xlabel('Principal Component 1')
axs[0].set_ylabel('Principal Component 2')

# Plot LDA result
scatter = axs[1].scatter(X_lda[:, 0], X_lda[:, 1], c=y, cmap='viridis')
legend2 = axs[1].legend(*scatter.legend_elements(), title="Classes")
axs[1].add_artist(legend2)
axs[1].set_title('LDA of Digits Dataset with Selected Features')
axs[1].set_xlabel('Linear Discriminant 1')
axs[1].set_ylabel('Linear Discriminant 2')

# Display plots
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_digits
import matplotlib.pyplot as plt

# Load dataset
data = load_digits()
X = data.data
y = data.target

# Standardize the data
scaler = StandardScaler()
X_centered = scaler.fit_transform(X)

# Apply classical PCA and SVD for comparison
n_components = 2
pca = PCA(n_components=n_components)
X_pca = pca.fit_transform(X_centered)

U, S, Vt = np.linalg.svd(X_centered, full_matrices=False)
U_reduced = U[:, :n_components]
S_reduced = S[:n_components]
Vt_reduced = Vt[:n_components, :]

# Create subplots (modified to only create 2 subplots)
fig, axs = plt.subplots(1, 2, figsize=(18, 6))

# Plot Quantum PCA result (same as before)
scatter = axs[0].scatter(X_qpca[:, 0], X_qpca[:, 1], c=y, cmap='viridis')
legend1 = axs[0].legend(*scatter.legend_elements(), title="Classes")
axs[0].add_artist(legend1)
axs[0].set_title('Quantum PCA of Digits Dataset')
axs[0].set_xlabel('Principal Component 1')
axs[0].set_ylabel('Principal Component 2')

# Plot Quantum SVD result (same as before)
scatter = axs[1].scatter(U_qsvd[:, 0], U_qsvd[:, 1], c=y, cmap='viridis')
legend2 = axs[1].legend(*scatter.legend_elements(), title="Classes")
axs[1].add_artist(legend2)
axs[1].set_title('Quantum SVD of Digits Dataset')
axs[1].set_xlabel('Singular Value 1')
axs[1].set_ylabel('Singular Value 2')

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
from scipy.linalg import logm, eigh
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_digits
import matplotlib.pyplot as plt

# Function to compute the matrix logarithm with numerical stability
def matrix_logarithm(matrix):
    eigenvalues, eigenvectors = eigh(matrix)
    eigenvalues = np.maximum(eigenvalues, 1e-10)  # Avoid log(0)
    log_eigenvalues = np.log(eigenvalues)
    return eigenvectors @ np.diag(log_eigenvalues) @ eigenvectors.T

# Accurate quantum relative entropy calculation
def quantum_relative_entropy(rho, sigma):
    epsilon = 1e-10
    
    # Ensure matrices are Hermitian and positive semi-definite
    rho = np.maximum(rho, epsilon * np.eye(rho.shape[0]))
    sigma = np.maximum(sigma, epsilon * np.eye(sigma.shape[0]))
    
    # Compute the logarithms of the matrices
    rho_log = matrix_logarithm(rho)
    sigma_log = matrix_logarithm(sigma)
    
    # Ensure the matrices have the same shape
    if rho.shape != sigma.shape:
        raise ValueError("Matrix dimensions do not match for entropy calculation.")
    
    # Compute quantum relative entropy
    rho_log_sigma = rho @ (rho_log - sigma_log)
    return np.trace(rho_log_sigma)

# Load dataset
data = load_digits()
X = data.data
y = data.target

# Standardize the data
scaler = StandardScaler()
X_centered = scaler.fit_transform(X)

# Number of features to select
k = 30

# Apply quantum filter method for feature selection
def quantum_filter_method(X, y, k):
    scores = np.array([f_quant(X[:, i], y) for i in range(X.shape[1])])
    ranked_features = np.argsort(scores)[::-1]  # Sort scores in descending order
    selected_features = ranked_features[:k]
    X_selected = X[:, selected_features]
    return X_selected, selected_features

def f_quant(X_col, y):
    # Simulated quantum feature scoring based on dot product
    return np.dot(X_col, np.random.rand(X_col.shape[0]))

X_selected, selected_features = quantum_filter_method(X_centered, y, k)

# Perform qPCA on the selected features
def qPCA(X, n_components):
    pca = PCA(n_components=n_components)
    X_qPCA = pca.fit_transform(X)
    return X_qPCA, pca.explained_variance_ratio_

n_components = 2
X_qPCA, qPCA_explained_variance_ratio = qPCA(X_selected, n_components)

# Perform qSVD on the selected features
def qSVD(X, n_components):
    U, S, Vt = np.linalg.svd(X, full_matrices=False)
    return U[:, :n_components], S[:n_components], Vt[:n_components, :]

U_qSVD, S_qSVD, Vt_qSVD = qSVD(X_selected, n_components)

# Reconstruct the data using qSVD
def matrix_reconstruction(U, S, Vt, k):
    S_full = np.zeros_like(S)
    S_full[:k] = S
    return U @ np.diag(S_full) @ Vt

X_reconstructed = matrix_reconstruction(U_qSVD, S_qSVD, Vt_qSVD, n_components)

# Calculate reconstruction error
reconstruction_error = np.linalg.norm(X_selected - X_reconstructed, axis=1)

# Compute covariance matrices
rho = np.cov(X_centered.T)
sigma = np.cov(X_selected.T)

# Ensure rho and sigma are square matrices and match in size
if rho.shape != sigma.shape:
    min_shape = min(rho.shape[0], sigma.shape[0])
    rho = rho[:min_shape, :min_shape]
    sigma = sigma[:min_shape, :min_shape]

# Calculate Quantum Relative Entropy
quantum_entropy = quantum_relative_entropy(rho, sigma)

# Create subplots
fig, axs = plt.subplots(2, 3, figsize=(18, 12))

# Plot feature selection results (Example: feature importance)
axs[0, 0].bar(range(len(selected_features)), [f_quant(X[:, i], y) for i in selected_features])
axs[0, 0].set_title('Feature Importance (Quantum Scoring)')
axs[0, 0].set_xlabel('Feature Index')
axs[0, 0].set_ylabel('Score')

# Plot qPCA result
scatter_qPCA = axs[0, 1].scatter(X_qPCA[:, 0], X_qPCA[:, 1], c=y, cmap='viridis')
legend1_qPCA = axs[0, 1].legend(*scatter_qPCA.legend_elements(), title="Classes")
axs[0, 1].add_artist(legend1_qPCA)
axs[0, 1].set_title('qPCA of Selected Features')
axs[0, 1].set_xlabel('Principal Component 1')
axs[0, 1].set_ylabel('Principal Component 2')

# Plot enhanced qSVD singular values
axs[0, 2].bar(range(len(S_qSVD)), S_qSVD)
axs[0, 2].set_title('qSVD Singular Values')
axs[0, 2].set_xlabel('Index')
axs[0, 2].set_ylabel('Singular Value')

# Plot LDA result (for comparison)
X_lda = lda(X_selected, y, n_components)

scatter_lda = axs[1, 0].scatter(X_lda[:, 0], X_lda[:, 1], c=y, cmap='viridis')
legend1_lda = axs[1, 0].legend(*scatter_lda.legend_elements(), title="Classes")
axs[1, 0].add_artist(legend1_lda)
axs[1, 0].set_title('LDA of Selected Features')
axs[1, 0].set_xlabel('Linear Discriminant 1')
axs[1, 0].set_ylabel('Linear Discriminant 2')

# Plot Reconstruction Error Distribution
axs[1, 1].hist(reconstruction_error, bins=30, edgecolor='k', alpha=0.7)
axs[1, 1].set_title('Reconstruction Error Distribution')
axs[1, 1].set_xlabel('Reconstruction Error')
axs[1, 1].set_ylabel('Frequency')

# Plot Quantum Relative Entropy
axs[1, 2].bar(['Entropy'], [quantum_entropy])
axs[1, 2].set_title('Quantum Relative Entropy')
axs[1, 2].set_ylabel('Entropy Value')

plt.tight_layout()
plt.savefig("f3.svg")
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.datasets import load_iris, load_digits, load_wine
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD

# Load and preprocess datasets
def load_and_preprocess_dataset(dataset_name):
    if dataset_name == 'iris':
        data = load_iris()
    elif dataset_name == 'digits':
        data = load_digits()
    elif dataset_name == 'wine':
        data = load_wine()
    elif dataset_name == 'mnist':
        data = fetch_openml('mnist_784')
        data.data, data.target = np.array(data.data), np.array(data.target)
    elif dataset_name == 'cifar10':
        # CIFAR-10 is large, you might want to use a subset for demonstration
        # Example: Load 10,000 samples from CIFAR-10 dataset
        data = fetch_openml('cifar_10')
        data.data, data.target = np.array(data.data[:10000]), np.array(data.target[:10000])
    else:
        raise ValueError("Unknown dataset name")

    X, y = data.data, data.target
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    return X_scaled, y

# Quantum Filter Method
def quantum_filter_method(X, y, k):
    scores = np.array([f_quant(X[:, i], y) for i in range(X.shape[1])])
    ranked_features = np.argsort(scores)[::-1]
    selected_features = ranked_features[:k]
    X_selected = X[:, selected_features]
    return X_selected, selected_features

def f_quant(X_col, y):
    return np.dot(X_col, np.random.rand(X_col.shape[0]))

def pca(X, n_components):
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X)
    return X_pca, pca.explained_variance_ratio_

def qSVD(X, n_components):
    U, S, Vt = np.linalg.svd(X, full_matrices=False)
    return U[:, :n_components], S[:n_components], Vt[:n_components, :]

def matrix_reconstruction(U, S, Vt, k):
    S_full = np.zeros_like(S)
    S_full[:k] = S
    return U @ np.diag(S_full) @ Vt

def quantum_relative_entropy(rho, sigma):
    epsilon = 1e-10
    rho = np.maximum(rho, epsilon * np.eye(rho.shape[0]))
    sigma = np.maximum(sigma, epsilon * np.eye(sigma.shape[0]))
    rho_log = matrix_logarithm(rho)
    sigma_log = matrix_logarithm(sigma)
    rho_log_sigma = rho @ (rho_log - sigma_log)
    return np.trace(rho_log_sigma)

def matrix_logarithm(matrix):
    from scipy.linalg import eigh
    eigenvalues, eigenvectors = eigh(matrix)
    eigenvalues = np.maximum(eigenvalues, 1e-10)
    log_eigenvalues = np.log(eigenvalues)
    return eigenvectors @ np.diag(log_eigenvalues) @ eigenvectors.T

# Define a function to run analysis for each dataset
def run_analysis(dataset_name, k=30, n_components=2):
    X, y = load_and_preprocess_dataset(dataset_name)
    
    X_selected, selected_features = quantum_filter_method(X, y, k)
    X_qPCA, qPCA_explained_variance_ratio = pca(X_selected, n_components)
    
    U_qSVD, S_qSVD, Vt_qSVD = qSVD(X_selected, n_components)
    X_reconstructed = matrix_reconstruction(U_qSVD, S_qSVD, Vt_qSVD, n_components)
    reconstruction_error = np.linalg.norm(X_selected - X_reconstructed, axis=1)
    
    rho = np.cov(X.T)
    sigma = np.cov(X_selected.T)
    if rho.shape != sigma.shape:
        min_shape = min(rho.shape[0], sigma.shape[0])
        rho = rho[:min_shape, :min_shape]
        sigma = sigma[:min_shape, :min_shape]
    quantum_entropy = quantum_relative_entropy(rho, sigma)
    
    # Plotting
    fig, axs = plt.subplots(2, 3, figsize=(18, 12))

    axs[0, 0].bar(range(len(selected_features)), [f_quant(X[:, i], y) for i in selected_features])
    axs[0, 0].set_title(f'{dataset_name} - Feature Importance (Quantum Scoring)')
    axs[0, 0].set_xlabel('Feature Index')
    axs[0, 0].set_ylabel('Score')

    scatter_qPCA = axs[0, 1].scatter(X_qPCA[:, 0], X_qPCA[:, 1], c=y, cmap='viridis')
    legend1_qPCA = axs[0, 1].legend(*scatter_qPCA.legend_elements(), title="Classes")
    axs[0, 1].add_artist(legend1_qPCA)
    axs[0, 1].set_title(f'{dataset_name} - qPCA Result')
    axs[0, 1].set_xlabel('Principal Component 1')
    axs[0, 1].set_ylabel('Principal Component 2')

    axs[0, 2].bar(range(len(S_qSVD)), S_qSVD)
    axs[0, 2].set_title(f'{dataset_name} - qSVD Singular Values')
    axs[0, 2].set_xlabel('Index')
    axs[0, 2].set_ylabel('Singular Value')

    scatter_lda = axs[1, 0].scatter(X_selected[:, 0], X_selected[:, 1], c=y, cmap='viridis')
    legend1_lda = axs[1, 0].legend(*scatter_lda.legend_elements(), title="Classes")
    axs[1, 0].add_artist(legend1_lda)
    axs[1, 0].set_title(f'{dataset_name} - LDA Result')
    axs[1, 0].set_xlabel('Linear Discriminant 1')
    axs[1, 0].set_ylabel('Linear Discriminant 2')

    axs[1, 1].hist(reconstruction_error, bins=30, edgecolor='k', alpha=0.7)
    axs[1, 1].set_title(f'{dataset_name} - Reconstruction Error Distribution')
    axs[1, 1].set_xlabel('Reconstruction Error')
    axs[1, 1].set_ylabel('Frequency')

    axs[1, 2].bar(['Entropy'], [quantum_entropy])
    axs[1, 2].set_title(f'{dataset_name} - Quantum Relative Entropy')
    axs[1, 2].set_ylabel('Entropy Value')

    plt.tight_layout()
    plt.savefig("f4.svg")
    plt.show()

# Run the analysis for different datasets
for dataset in ['iris', 'digits', 'wine', 'mnist', 'cifar10']:
    run_analysis(dataset)


In [ ]:
# Install required libraries
!pip install opencv-python scikit-learn matplotlib numpy

# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.manifold import TSNE
from torchvision import datasets, transforms
import cv2

# Function to load MNIST dataset
def load_mnist_data():
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
    trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
    return trainset

# Function to load Lenna image
def load_lenna_image():
    lenna_image = cv2.imread('path/to/lenna_256.jpg', cv2.IMREAD_GRAYSCALE)
    lenna_image = cv2.resize(lenna_image, (28, 28))
    return lenna_image / 255.0  # Normalize the image

# Load data
mnist_data = load_mnist_data()
lenna_image = load_lenna_image()

# Sample MNIST image
sample_mnist_image, _ = mnist_data[0]
sample_mnist_image = sample_mnist_image.numpy()[0]
def perform_pca(data, n_components):
    pca = PCA(n_components=n_components)
    data_centered = data - np.mean(data, axis=0)
    transformed_data = pca.fit_transform(data_centered)
    return transformed_data, pca

# Reshape images to 1D arrays and stack
data_matrix = np.vstack([sample_mnist_image.flatten(), lenna_image.flatten()])

# Perform PCA
transformed_data, pca = perform_pca(data_matrix, n_components=2)

# Visualize PCA results
plt.figure(figsize=(8, 6))
plt.scatter(transformed_data[:, 0], transformed_data[:, 1], marker='o')
plt.title('PCA on MNIST and Lenna Images')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.show()
def perform_lda(data, labels, n_components):
    lda = LDA(n_components=n_components)
    transformed_data = lda.fit_transform(data, labels)
    return transformed_data, lda

# Create dummy labels for MNIST and Lenna images
labels = np.array([0, 1])

# Perform LDA
transformed_lda_data, lda = perform_lda(data_matrix, labels, n_components=1)

# Visualize LDA results
plt.figure(figsize=(8, 6))
plt.scatter(transformed_lda_data[:, 0], np.zeros_like(transformed_lda_data[:, 0]), marker='o')
plt.title('LDA on MNIST and Lenna Images')
plt.xlabel('Linear Discriminant 1')
plt.show()
def perform_tsne(data, n_components, perplexity):
    tsne = TSNE(n_components=n_components, perplexity=perplexity, n_iter=300)
    transformed_data = tsne.fit_transform(data)
    return transformed_data

# Perform t-SNE
transformed_tsne_data = perform_tsne(data_matrix, n_components=2, perplexity=2)

# Visualize t-SNE results
plt.figure(figsize=(8, 6))
plt.scatter(transformed_tsne_data[:, 0], transformed_tsne_data[:, 1], marker='o')
plt.title('t-SNE on MNIST and Lenna Images')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.show()


In [ ]:
# Install required libraries
!pip install opencv-python scikit-learn matplotlib numpy

# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.manifold import TSNE
from torchvision import datasets, transforms
import cv2

# Function to load MNIST dataset
def load_mnist_data():
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
    trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
    return trainset

# Function to load Lenna image
def load_lenna_image():
    lenna_image = cv2.imread('path/to/lenna_256.jpg', cv2.IMREAD_GRAYSCALE)
    lenna_image = cv2.resize(lenna_image, (28, 28))
    return lenna_image / 255.0  # Normalize the image

# Load data
mnist_data = load_mnist_data()
lenna_image = load_lenna_image()

# Sample MNIST image
sample_mnist_image, _ = mnist_data[0]
sample_mnist_image = sample_mnist_image.numpy()[0]
def perform_pca(data, n_components):
    pca = PCA(n_components=n_components)
    data_centered = data - np.mean(data, axis=0)
    transformed_data = pca.fit_transform(data_centered)
    return transformed_data, pca

# Reshape images to 1D arrays and stack
data_matrix = np.vstack([sample_mnist_image.flatten(), lenna_image.flatten()])

# Perform PCA
transformed_data, pca = perform_pca(data_matrix, n_components=2)

# Visualize PCA results
plt.figure(figsize=(8, 6))
plt.scatter(transformed_data[:, 0], transformed_data[:, 1], marker='o')
plt.title('PCA on MNIST and Lenna Images')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.show()
def perform_lda(data, labels, n_components):
    lda = LDA(n_components=n_components)
    transformed_data = lda.fit_transform(data, labels)
    return transformed_data, lda

# Create dummy labels for MNIST and Lenna images
labels = np.array([0, 1])

# Perform LDA
transformed_lda_data, lda = perform_lda(data_matrix, labels, n_components=1)

# Visualize LDA results
plt.figure(figsize=(8, 6))
plt.scatter(transformed_lda_data[:, 0], np.zeros_like(transformed_lda_data[:, 0]), marker='o')
plt.title('LDA on MNIST and Lenna Images')
plt.xlabel('Linear Discriminant 1')
plt.show()
def perform_tsne(data, n_components, perplexity):
    tsne = TSNE(n_components=n_components, perplexity=perplexity, n_iter=300)
    transformed_data = tsne.fit_transform(data)
    return transformed_data

# Perform t-SNE
transformed_tsne_data = perform_tsne(data_matrix, n_components=2, perplexity=2)

# Visualize t-SNE results
plt.figure(figsize=(8, 6))
plt.scatter(transformed_tsne_data[:, 0], transformed_tsne_data[:, 1], marker='o')
plt.title('t-SNE on MNIST and Lenna Images')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.show()
